In [1]:
%%capture
!pip install -U xgboost
!pip install -U polars
!pip install -U optuna
!pip install -U catboost
!pip install -U lightgbm

In [2]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import time
import xgboost as xgb
import catboost
import lightgbm as lgb
import optuna
import pandas as pd
from scipy.stats import spearmanr
from scipy.optimize import minimize

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load data
train = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/train.parquet').drop('__index_level_0__')
test = pl.read_parquet('/kaggle/input/aeroclub-recsys-2025/test.parquet').drop('__index_level_0__').with_columns(pl.lit(0, dtype=pl.Int64).alias("selected"))

data_raw = pl.concat((train, test))

## Helpers

In [4]:
def hitrate_at_3(y_true, y_pred, groups):
    df = pl.DataFrame({
        'group': groups,
        'pred': y_pred,
        'true': y_true
    })
    
    return (
        df.filter(pl.col("group").count().over("group") > 10)
        .sort(["group", "pred"], descending=[False, True])
        .group_by("group", maintain_order=True)
        .head(3)
        .group_by("group")
        .agg(pl.col("true").max())
        .select(pl.col("true").mean())
        .item()
    )

## Feature Engineering

In [5]:
df = data_raw.clone()

# More efficient duration to minutes converter
def dur_to_min(col):
    # Extract days and time parts in one pass
    days = col.str.extract(r"^(\d+)\.", 1).cast(pl.Int64).fill_null(0) * 1440
    time_str = pl.when(col.str.contains(r"^\d+\.")).then(col.str.replace(r"^\d+\.", "")).otherwise(col)
    hours = time_str.str.extract(r"^(\d+):", 1).cast(pl.Int64).fill_null(0) * 60
    minutes = time_str.str.extract(r":(\d+):", 1).cast(pl.Int64).fill_null(0)
    return (days + hours + minutes).fill_null(0)

# Process duration columns
dur_cols = ["legs0_duration", "legs1_duration"] + [f"legs{l}_segments{s}_duration" for l in (0, 1) for s in (0, 1)]
dur_exprs = [dur_to_min(pl.col(c)).alias(c) for c in dur_cols if c in df.columns]

# Apply duration transformations first
if dur_exprs:
    df = df.with_columns(dur_exprs)

# Precompute marketing carrier columns check
mc_cols = [f'legs{l}_segments{s}_marketingCarrier_code' for l in (0, 1) for s in range(4)]
mc_exists = [col for col in mc_cols if col in df.columns]

# Combine all initial transformations
df = df.with_columns([
        # Price features
        (pl.col("totalPrice") / (pl.col("taxes") + 1)).alias("price_per_tax"),
        (pl.col("taxes") / (pl.col("totalPrice") + 1)).alias("tax_rate"),
        pl.col("totalPrice").log1p().alias("log_price"),
        
        # Duration features
        (pl.col("legs0_duration").fill_null(0) + pl.col("legs1_duration").fill_null(0)).alias("total_duration"),
        pl.when(pl.col("legs1_duration").fill_null(0) > 0)
            .then(pl.col("legs0_duration") / (pl.col("legs1_duration") + 1))
            .otherwise(1.0).alias("duration_ratio"),
        
        # Trip type
        (pl.col("legs1_duration").is_null() | 
         (pl.col("legs1_duration") == 0) | 
         pl.col("legs1_segments0_departureFrom_airport_iata").is_null()).cast(pl.Int32).alias("is_one_way"),
        
        # Total segments count
        (pl.sum_horizontal(pl.col(col).is_not_null().cast(pl.UInt8) for col in mc_exists) 
         if mc_exists else pl.lit(0)).alias("l0_seg"),
        
        # FF features
        (pl.col("frequentFlyer").fill_null("").str.count_matches("/") + 
         (pl.col("frequentFlyer").fill_null("") != "").cast(pl.Int32)).alias("n_ff_programs"),
        
        # Binary features
        pl.col("corporateTariffCode").is_not_null().cast(pl.Int32).alias("has_corporate_tariff"),
        (pl.col("pricingInfo_isAccessTP") == 1).cast(pl.Int32).alias("has_access_tp"),
        
        # Baggage & fees
        (pl.col("legs0_segments0_baggageAllowance_quantity").fill_null(0) + 
         pl.col("legs1_segments0_baggageAllowance_quantity").fill_null(0)).alias("baggage_total"),
        (pl.col("miniRules0_monetaryAmount").fill_null(0) + 
         pl.col("miniRules1_monetaryAmount").fill_null(0)).alias("total_fees"),
        
        # Routes & carriers
        pl.col("searchRoute").is_in(["MOWLED/LEDMOW", "LEDMOW/MOWLED", "MOWLED", "LEDMOW", "MOWAER/AERMOW"])
            .cast(pl.Int32).alias("is_popular_route"),
        
        # Cabin
        pl.mean_horizontal(["legs0_segments0_cabinClass", "legs1_segments0_cabinClass"]).alias("avg_cabin_class"),
        (pl.col("legs0_segments0_cabinClass").fill_null(0) - 
         pl.col("legs1_segments0_cabinClass").fill_null(0)).alias("cabin_class_diff"),
])

# Segment counts - more efficient
seg_exprs = []
for leg in (0, 1):
    seg_cols = [f"legs{leg}_segments{s}_duration" for s in range(4) if f"legs{leg}_segments{s}_duration" in df.columns]
    if seg_cols:
        seg_exprs.append(
            pl.sum_horizontal(pl.col(c).is_not_null() for c in seg_cols)
                .cast(pl.Int32).alias(f"n_segments_leg{leg}")
        )
    else:
        seg_exprs.append(pl.lit(0).cast(pl.Int32).alias(f"n_segments_leg{leg}"))

# Add segment-based features
# First create segment counts
df = df.with_columns(seg_exprs)

# Then use them for derived features
df = df.with_columns([
    (pl.col("n_segments_leg0") + pl.col("n_segments_leg1")).alias("total_segments"),
    (pl.col("n_segments_leg0") == 1).cast(pl.Int32).alias("is_direct_leg0"),
    pl.when(pl.col("is_one_way") == 1).then(0)
        .otherwise((pl.col("n_segments_leg1") == 1).cast(pl.Int32)).alias("is_direct_leg1"),
])

# More derived features
df = df.with_columns([
    (pl.col("is_direct_leg0") & pl.col("is_direct_leg1")).cast(pl.Int32).alias("both_direct"),
    ((pl.col("isVip") == 1) | (pl.col("n_ff_programs") > 0)).cast(pl.Int32).alias("is_vip_freq"),
    (pl.col("baggage_total") > 0).cast(pl.Int32).alias("has_baggage"),
    (pl.col("total_fees") > 0).cast(pl.Int32).alias("has_fees"),
    (pl.col("total_fees") / (pl.col("totalPrice") + 1)).alias("fee_rate"),
    pl.col("Id").count().over("ranker_id").alias("group_size"),
])

# Add major carrier flag if column exists
if "legs0_segments0_marketingCarrier_code" in df.columns:
    df = df.with_columns(
        pl.col("legs0_segments0_marketingCarrier_code").is_in(["SU", "S7", "U6"])
            .cast(pl.Int32).alias("is_major_carrier")
    )
else:
    df = df.with_columns(pl.lit(0).alias("is_major_carrier"))

df = df.with_columns(pl.col("group_size").log1p().alias("group_size_log"))

# Time features - batch process
time_exprs = []
for col in ("legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"):
    if col in df.columns:
        dt = pl.col(col).str.to_datetime(strict=False)
        h = dt.dt.hour().fill_null(12)
        time_exprs.extend([
            h.alias(f"{col}_hour"),
            dt.dt.weekday().fill_null(0).alias(f"{col}_weekday"),
            (((h >= 6) & (h <= 9)) | ((h >= 17) & (h <= 20))).cast(pl.Int32).alias(f"{col}_business_time")
        ])
if time_exprs:
    df = df.with_columns(time_exprs)

# Batch rank computations - more efficient with single pass
# First apply the columns that will be used for ranking
df = df.with_columns([
    pl.col("group_size").log1p().alias("group_size_log"),
])

# Price and duration basic ranks
rank_exprs = []
for col, alias in [("totalPrice", "price"), ("total_duration", "duration")]:
    rank_exprs.append(pl.col(col).rank().over("ranker_id").alias(f"{alias}_rank"))

# Price-specific features
price_exprs = [
    (pl.col("totalPrice").rank("average").over("ranker_id") / 
     pl.col("totalPrice").count().over("ranker_id")).alias("price_pct_rank"),
    (pl.col("totalPrice") == pl.col("totalPrice").min().over("ranker_id")).cast(pl.Int32).alias("is_cheapest"),
    ((pl.col("totalPrice") - pl.col("totalPrice").median().over("ranker_id")) / 
     (pl.col("totalPrice").std().over("ranker_id") + 1)).alias("price_from_median"),
    (pl.col("l0_seg") == pl.col("l0_seg").min().over("ranker_id")).cast(pl.Int32).alias("is_min_segments"),
]

# Apply initial ranks
df = df.with_columns(rank_exprs + price_exprs)

# Cheapest direct - more efficient
direct_cheapest = (
    df.filter(pl.col("is_direct_leg0") == 1)
    .group_by("ranker_id")
    .agg(pl.col("totalPrice").min().alias("min_direct"))
)

df = df.join(direct_cheapest, on="ranker_id", how="left").with_columns(
    ((pl.col("is_direct_leg0") == 1) & 
     (pl.col("totalPrice") == pl.col("min_direct"))).cast(pl.Int32).fill_null(0).alias("is_direct_cheapest")
).drop("min_direct")

In [6]:
# Advanced Feature Engineering - Focus on Business Traveler Preferences
print("Adding advanced business traveler features...")

# 1. Enhanced price and policy features
df = df.with_columns([
    # Corporate policy compliance
    (pl.col("pricingInfo_isAccessTP") == 1).cast(pl.Int32).alias("policy_compliant"),
    
    # Enhanced price buckets with controlled group stats
    pl.when(pl.col("price_pct_rank") <= 0.15).then(1)
    .when(pl.col("price_pct_rank") <= 0.3).then(2)
    .when(pl.col("price_pct_rank") <= 0.5).then(3)
    .when(pl.col("price_pct_rank") <= 0.7).then(4)
    .when(pl.col("price_pct_rank") <= 0.85).then(5)
    .otherwise(6).alias("price_bucket"),
    
    # Price competitiveness (keep essential group features)
    (pl.col("totalPrice") <= pl.col("totalPrice").quantile(0.25).over("ranker_id")).cast(pl.Int32).alias("is_cheap_quartile"),
    (pl.col("totalPrice") == pl.col("totalPrice").min().over("ranker_id")).cast(pl.Int32).alias("is_absolute_cheapest"),
    
    # Tax efficiency for business
    pl.when(pl.col("taxes") > 0).then(pl.col("totalPrice") / pl.col("taxes")).otherwise(0).alias("price_tax_efficiency"),
])

# 2. Advanced time features for business travelers
time_features = []
for prefix in ["legs0_departureAt", "legs0_arrivalAt", "legs1_departureAt", "legs1_arrivalAt"]:
    if f"{prefix}_hour" in df.columns:
        time_features.extend([
            # Premium business hours
            ((pl.col(f"{prefix}_hour") >= 7) & (pl.col(f"{prefix}_hour") <= 9)).cast(pl.Int32).alias(f"{prefix}_morning_business"),
            ((pl.col(f"{prefix}_hour") >= 17) & (pl.col(f"{prefix}_hour") <= 19)).cast(pl.Int32).alias(f"{prefix}_evening_business"),
            # Avoid red-eye flights
            ((pl.col(f"{prefix}_hour") >= 23) | (pl.col(f"{prefix}_hour") <= 5)).cast(pl.Int32).alias(f"{prefix}_red_eye"),
            # Premium time slots
            (pl.col(f"{prefix}_hour").is_in([7, 8, 9, 18, 19, 20])).cast(pl.Int32).alias(f"{prefix}_premium_time"),
            # Weekend patterns
            ((pl.col(f"{prefix}_weekday") >= 5) & (pl.col(f"{prefix}_hour") >= 10)).cast(pl.Int32).alias(f"{prefix}_weekend_leisure"),
        ])

if time_features:
    df = df.with_columns(time_features)

# 3. Seat availability and scarcity features
seat_features = []
for leg in [0, 1]:
    for seg in [0, 1]:
        seat_col = f"legs{leg}_segments{seg}_seatsAvailable"
        if seat_col in df.columns:
            seat_features.extend([
                # Seat scarcity (low availability premium)
                (pl.col(seat_col).fill_null(0) <= 5).cast(pl.Int32).alias(f"{seat_col}_scarce"),
                # High availability
                (pl.col(seat_col).fill_null(0) >= 20).cast(pl.Int32).alias(f"{seat_col}_abundant"),
                # Normalized seat availability within group
                (pl.col(seat_col) / (pl.col(seat_col).mean().over("ranker_id") + 1)).alias(f"{seat_col}_relative"),
            ])

if seat_features:
    df = df.with_columns(seat_features)

# 4. Cancellation and change policy features (critical for business)
policy_features = []
has_cancellation = False
has_exchange = False

# Check if cancellation features exist
if "miniRules0_monetaryAmount" in df.columns:
    has_cancellation = True
    policy_features.extend([
        # Flexible cancellation
        (pl.col("miniRules0_monetaryAmount").fill_null(999999) == 0).cast(pl.Int32).alias("free_cancellation"),
        # Low cancellation fee
        (pl.col("miniRules0_monetaryAmount").fill_null(999999) <= pl.col("totalPrice") * 0.1).cast(pl.Int32).alias("low_cancel_fee"),
        # Cancellation fee rate
        (pl.col("miniRules0_monetaryAmount") / (pl.col("totalPrice") + 1)).alias("cancel_fee_rate"),
    ])

if "miniRules1_monetaryAmount" in df.columns:
    has_exchange = True
    policy_features.extend([
        # Flexible exchange
        (pl.col("miniRules1_monetaryAmount").fill_null(999999) == 0).cast(pl.Int32).alias("free_exchange"),
        # Low exchange fee
        (pl.col("miniRules1_monetaryAmount").fill_null(999999) <= pl.col("totalPrice") * 0.1).cast(pl.Int32).alias("low_exchange_fee"),
        # Exchange fee rate
        (pl.col("miniRules1_monetaryAmount") / (pl.col("totalPrice") + 1)).alias("exchange_fee_rate"),
    ])

if policy_features:
    df = df.with_columns(policy_features)

# 5. Route and carrier sophistication
route_features = []
has_airports = False
has_carriers = False

if "legs0_segments0_departureFrom_airport_iata" in df.columns:
    has_airports = True
    route_features.extend([
        # Major hub airports (Moscow, St. Petersburg)
        pl.col("legs0_segments0_departureFrom_airport_iata").is_in(["SVO", "DME", "VKO", "LED", "PKC"])
        .cast(pl.Int32).alias("departs_from_major_hub"),
        
        # International airports
        pl.col("legs0_segments0_arrivalTo_airport_iata").is_in(["LED", "SVO", "DME", "VKO"])
        .cast(pl.Int32).alias("arrives_to_major_hub"),
        
        # Airport consistency (same departure/arrival airports for round trips)
        (pl.col("legs0_segments0_departureFrom_airport_iata") == 
         pl.col("legs1_segments0_arrivalTo_airport_iata").fill_null("")).cast(pl.Int32).alias("consistent_airports"),
    ])

if "legs0_segments0_marketingCarrier_code" in df.columns:
    has_carriers = True
    route_features.extend([
        # Carrier consistency across legs
        (pl.col("legs0_segments0_marketingCarrier_code") == 
         pl.col("legs1_segments0_marketingCarrier_code").fill_null("")).cast(pl.Int32).alias("same_carrier_both_legs"),
        
        # Premium carriers (Aeroflot, S7, etc.)
        pl.col("legs0_segments0_marketingCarrier_code").is_in(["SU", "S7", "U6", "DP"])
        .cast(pl.Int32).alias("is_premium_carrier"),
        
        # Marketing vs Operating carrier alignment
        (pl.col("legs0_segments0_marketingCarrier_code") == 
         pl.col("legs0_segments0_operatingCarrier_code").fill_null("")).cast(pl.Int32).alias("aligned_carriers_leg0"),
    ])

if route_features:
    df = df.with_columns(route_features)

# 6. Aircraft and service quality features
aircraft_features = []
has_aircraft = False

if "legs0_segments0_aircraft_code" in df.columns:
    has_aircraft = True
    aircraft_features.extend([
        # Wide-body aircraft (better for long flights)
        pl.col("legs0_segments0_aircraft_code").is_in(["330", "777", "787", "320", "321"])
        .cast(pl.Int32).alias("wide_body_leg0"),
        
        # Modern aircraft
        pl.col("legs0_segments0_aircraft_code").is_in(["787", "350", "320", "321", "737"])
        .cast(pl.Int32).alias("modern_aircraft_leg0"),
    ])

if aircraft_features:
    df = df.with_columns(aircraft_features)

# 7. Cabin class optimization
cabin_features = []
has_cabin = False

if "legs0_segments0_cabinClass" in df.columns:
    has_cabin = True
    cabin_features.extend([
        # Business class upgrade availability
        (pl.col("legs0_segments0_cabinClass").fill_null(1) >= 2).cast(pl.Int32).alias("business_class_leg0"),
        # Premium economy or better
        (pl.col("legs0_segments0_cabinClass").fill_null(1) >= 1.5).cast(pl.Int32).alias("premium_class_leg0"),
        # Cabin class consistency
        (pl.col("legs0_segments0_cabinClass") == pl.col("legs1_segments0_cabinClass")).cast(pl.Int32).alias("consistent_cabin"),
    ])

if cabin_features:
    df = df.with_columns(cabin_features)

# 8. Competitive analysis within group (basic features first)
competitive_features = [
    # Direct flight premium
    pl.col("both_direct").sum().over("ranker_id").alias("n_direct_options"),
    
    # Premium combinations
    ((pl.col("both_direct") == 1) & (pl.col("is_cheap_quartile") == 1)).cast(pl.Int32).alias("is_cheap_direct"),
    ((pl.col("policy_compliant") == 1) & (pl.col("is_cheap_quartile") == 1)).cast(pl.Int32).alias("compliant_and_cheap"),
    
    # Business convenience
    ((pl.col("legs0_departureAt_hour").fill_null(12) >= 7) & 
     (pl.col("legs0_departureAt_hour").fill_null(12) <= 9)).cast(pl.Int32).alias("morning_departure_business"),
    
    # Fast and convenient
    (pl.col("total_duration") <= pl.col("total_duration").quantile(0.3).over("ranker_id")).cast(pl.Int32).alias("is_fast_option"),
]

df = df.with_columns(competitive_features)

# 8b. Advanced competitive features (requires previous features to exist)
advanced_competitive = []

if has_carriers:
    advanced_competitive.extend([
        # VIP treatment with premium carriers
        ((pl.col("is_vip_freq") == 1) & (pl.col("is_premium_carrier") == 1)).cast(pl.Int32).alias("vip_premium_carrier"),
    ])

if has_cancellation and has_carriers:
    advanced_competitive.extend([
        # Flexible and premium
        ((pl.col("free_cancellation") == 1) & (pl.col("is_premium_carrier") == 1)).cast(pl.Int32).alias("flexible_premium"),
    ])

if advanced_competitive:
    df = df.with_columns(advanced_competitive)

# 9. High-impact interaction features (safe with all previous features defined)
interaction_features = [
    # Price-policy compliance
    (pl.col("price_bucket") * pl.col("policy_compliant")).alias("price_policy_interaction"),
    
    # Business convenience score
    (pl.col("morning_departure_business") * pl.col("is_cheap_quartile") * pl.col("both_direct")).alias("business_value_score"),
    
    # Hub efficiency (basic version)
    (pl.col("both_direct") * pl.col("is_fast_option")).alias("hub_efficiency_score"),
    
    # Corporate optimization (basic version)
    (pl.col("policy_compliant") * pl.col("compliant_and_cheap")).alias("corporate_optimization"),
]

# Add premium service interaction if features exist
if has_carriers and has_cabin:
    interaction_features.append(
        (pl.col("is_premium_carrier") * pl.col("business_class_leg0")).alias("premium_service_score")
    )

# Add enhanced hub efficiency if airport features exist
if has_airports:
    interaction_features.append(
        (pl.col("departs_from_major_hub") * pl.col("both_direct") * pl.col("is_fast_option")).alias("enhanced_hub_efficiency_score")
    )

# Add enhanced corporate optimization if carrier features exist
if has_carriers:
    interaction_features.append(
        (pl.col("policy_compliant") * pl.col("compliant_and_cheap") * pl.col("same_carrier_both_legs")).alias("enhanced_corporate_optimization")
    )

df = df.with_columns(interaction_features)

print("Advanced business traveler features added successfully!")
print("Key improvements:")
print("- Corporate policy compliance and tax efficiency")
print("- Advanced time preferences for business travelers")
print("- Seat scarcity and availability features")
print("- Cancellation/exchange policy flexibility")
print("- Premium carrier and aircraft preferences")
print("- Hub airport and route optimization")
print("- Strategic business value interactions")
print("- Safe feature creation with proper dependency ordering")
print("- Targeting 0.5+ Kaggle score with business insights")

Adding advanced business traveler features...


Advanced business traveler features added successfully!
Key improvements:
- Corporate policy compliance and tax efficiency
- Advanced time preferences for business travelers
- Seat scarcity and availability features
- Cancellation/exchange policy flexibility
- Premium carrier and aircraft preferences
- Hub airport and route optimization
- Strategic business value interactions
- Safe feature creation with proper dependency ordering
- Targeting 0.5+ Kaggle score with business insights


In [7]:
# Fill nulls
data = df.with_columns(
    [pl.col(c).fill_null(0) for c in df.select(pl.selectors.numeric()).columns] +
    [pl.col(c).fill_null("missing") for c in df.select(pl.selectors.string()).columns]
)

## Feature Selection

In [8]:
# Categorical features
cat_features = [
    'nationality', 'searchRoute', 'corporateTariffCode',
    'bySelf', 'sex', 'companyID',
    # Leg 0 segments 0-1
    'legs0_segments0_aircraft_code', 'legs0_segments0_arrivalTo_airport_city_iata',
    'legs0_segments0_arrivalTo_airport_iata', 'legs0_segments0_departureFrom_airport_iata',
    'legs0_segments0_marketingCarrier_code', 'legs0_segments0_operatingCarrier_code',
    'legs0_segments0_flightNumber',
    'legs0_segments1_aircraft_code', 'legs0_segments1_arrivalTo_airport_city_iata',
    'legs0_segments1_arrivalTo_airport_iata', 'legs0_segments1_departureFrom_airport_iata',
    'legs0_segments1_marketingCarrier_code', 'legs0_segments1_operatingCarrier_code',
    'legs0_segments1_flightNumber',
    # Leg 1 segments 0-1
    'legs1_segments0_aircraft_code', 'legs1_segments0_arrivalTo_airport_city_iata',
    'legs1_segments0_arrivalTo_airport_iata', 'legs1_segments0_departureFrom_airport_iata',
    'legs1_segments0_marketingCarrier_code', 'legs1_segments0_operatingCarrier_code',
    'legs1_segments0_flightNumber',
    'legs1_segments1_aircraft_code', 'legs1_segments1_arrivalTo_airport_city_iata',
    'legs1_segments1_arrivalTo_airport_iata', 'legs1_segments1_departureFrom_airport_iata',
    'legs1_segments1_marketingCarrier_code', 'legs1_segments1_operatingCarrier_code',
    'legs1_segments1_flightNumber',
    # Only add price_bucket as new categorical
    'price_bucket'
]

# Columns to exclude (uninformative or problematic)
exclude_cols = [
    'Id', 'ranker_id', 'selected', 'profileId', 'requestDate',
    'legs0_departureAt', 'legs0_arrivalAt', 'legs1_departureAt', 'legs1_arrivalAt',
    'miniRules0_percentage', 'miniRules1_percentage',  # >90% missing
    'frequentFlyer',  # Already processed
    # Exclude constant columns
    'pricingInfo_passengerCount'
]

# Exclude segment 2-3 columns (>98% missing)
for leg in [0, 1]:
    for seg in [2, 3]:
        for suffix in ['aircraft_code', 'arrivalTo_airport_city_iata', 'arrivalTo_airport_iata',
                      'baggageAllowance_quantity', 'baggageAllowance_weightMeasurementType',
                      'cabinClass', 'departureFrom_airport_iata', 'duration', 'flightNumber',
                      'marketingCarrier_code', 'operatingCarrier_code', 'seatsAvailable']:
            exclude_cols.append(f'legs{leg}_segments{seg}_{suffix}')

feature_cols = [col for col in data.columns if col not in exclude_cols]
cat_features_final = [col for col in cat_features if col in feature_cols]

# Create CatBoost categorical feature indices (column positions in feature_cols)
catboost_cat_indices = [i for i, col in enumerate(feature_cols) if col in cat_features_final]

print(f"Using {len(feature_cols)} features ({len(cat_features_final)} categorical)")
print(f"CatBoost categorical indices: {len(catboost_cat_indices)} features")

X = data.select(feature_cols)
y = data.select('selected')
groups = data.select('ranker_id')

Using 180 features (35 categorical)
CatBoost categorical indices: 35 features


## Model Training and Tuning



### 1. XGBoost Model

In [10]:
data_xgb = X.with_columns([(pl.col(c).rank("dense") - 1).fill_null(-1).cast(pl.Int32) for c in cat_features_final])

n1 = 16487352 # split train to train and val (10%) in time
n2 = train.height
data_xgb_tr, data_xgb_va, data_xgb_te = data_xgb[:n1], data_xgb[n1:n2], data_xgb[n2:]
y_tr, y_va, y_te = y[:n1], y[n1:n2], y[n2:]
groups_tr, groups_va, groups_te = groups[:n1], groups[n1:n2], groups[n2:]

group_sizes_tr = groups_tr.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
group_sizes_va = groups_va.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
group_sizes_te = groups_te.group_by('ranker_id').agg(pl.len()).sort('ranker_id')['len'].to_numpy()
dtrain = xgb.DMatrix(data_xgb_tr, label=y_tr, group=group_sizes_tr, feature_names=data_xgb.columns)
dval   = xgb.DMatrix(data_xgb_va, label=y_va, group=group_sizes_va, feature_names=data_xgb.columns)
dtest  = xgb.DMatrix(data_xgb_te, label=y_te, group=group_sizes_te, feature_names=data_xgb.columns)

In [ ]:
# Optimized XGBoost parameters for business traveler features (v3 - Faster Training)
final_xgb_params = {
    'objective': 'rank:pairwise', 
    'eval_metric': 'ndcg@3', 
    'max_depth': 8,              # 保持深度8，平衡性能和速度
    'min_child_weight': 10,      # 保持10，良好的泛化
    'subsample': 0.92,           # 保持0.92，良好的泛化
    'colsample_bytree': 0.9,     # 保持0.9，充分利用特征
    'lambda': 3.0,              # 保持3.0，平衡正则化
    'alpha': 0.12,              # 保持0.12，良好的稀疏性
    'learning_rate': 0.065,     # 从0.055提升到0.065，更快收敛
    'gamma': 0.06,              # 保持0.06，适度的分裂要求
    'seed': RANDOM_STATE, 
    'n_jobs': -1,
    'tree_method': 'hist',
    'grow_policy': 'lossguide'
}

print("\nTraining XGBoost with v3 efficiency-optimized parameters...")
print("V3 efficiency optimizations (保持性能，提升速度):")
print("- Increased learning rate (0.065) for faster convergence")
print("- Reduced max iterations based on log analysis")
print("- Tighter early stopping for efficiency")
print("- Targeting same performance with 30% less iterations")

xgb_model = xgb.train(
    final_xgb_params, dtrain,
    num_boost_round=1200,        # 从1600减少到1200 (基于日志1042轮停止)
    evals=[(dtrain, 'train'), (dval, 'val')],
    early_stopping_rounds=80,    # 从110减少到80，更激进的停止
    verbose_eval=50
)


Training XGBoost with v2 business-optimized parameters...
V2 optimizations for better convergence:
- Reduced depth (8) to prevent slight overfitting
- Improved sampling rates for better generalization
- Fine-tuned regularization for optimal expressiveness
- Adjusted learning rate for faster convergence


[0]	train-ndcg@3:0.78564	val-ndcg@3:0.80987

[50]	train-ndcg@3:0.81576	val-ndcg@3:0.83376

[100]	train-ndcg@3:0.82366	val-ndcg@3:0.83657


[150]	train-ndcg@3:0.83067	val-ndcg@3:0.83838


...

[850]	train-ndcg@3:0.88546	val-ndcg@3:0.84451


[900]	train-ndcg@3:0.88773	val-ndcg@3:0.84500

[950]	train-ndcg@3:0.89052	val-ndcg@3:0.84522

[1000]	train-ndcg@3:0.89354	val-ndcg@3:0.84484

[1042]	train-ndcg@3:0.89669	val-ndcg@3:0.84534

### 3. LightGBM Model

In [12]:
# CODE CELL
# LightGBM requires its own Dataset object. We can reuse the rank-encoded data from XGBoost.
print("Creating LightGBM Datasets...")
lgb_train = lgb.Dataset(
    data=data_xgb_tr, 
    label=y_tr.to_numpy().flatten(), 
    group=group_sizes_tr,
    feature_name=feature_cols,
    free_raw_data=False
)

lgb_val = lgb.Dataset(
    data=data_xgb_va, 
    label=y_va.to_numpy().flatten(), 
    group=group_sizes_va,
    feature_name=feature_cols,
    reference=lgb_train,
    free_raw_data=False
)
print("LightGBM Datasets created successfully.")

Creating LightGBM Datasets...


LightGBM Datasets created successfully.


In [ ]:
# Optimized LightGBM parameters for business traveler features (v3 - Faster Training)
final_lgb_params = {
    'objective': 'lambdarank', 
    'metric': 'ndcg', 
    'boosting_type': 'gbdt',
    'eval_at': [3],
    'num_leaves': 140,           # 保持140，良好的模型容量
    'learning_rate': 0.18,       # 从0.15提升到0.18，更快收敛
    'min_child_samples': 60,     # 保持60，良好的泛化
    'lambda_l1': 0.006,          # 保持0.006，适度L1正则化
    'lambda_l2': 7.5,            # 保持7.5，良好的L2正则化
    'feature_fraction': 0.75,    # 保持0.75，防止过拟合
    'bagging_fraction': 0.84,    # 保持0.84，良好的采样
    'bagging_freq': 5,           
    'min_gain_to_split': 0.005,  # 保持0.005，适度的分裂要求
    'max_depth': 10,             # 保持10，控制复杂度
    'feature_pre_filter': False, # 修复LightGBM警告
    'force_row_wise': True,      
    'n_jobs': -1, 
    'random_state': RANDOM_STATE, 
    'label_gain': [0, 1],
    'verbosity': -1
}

print("\nTraining LightGBM model with v3 efficiency-optimized parameters...")
print("V3 efficiency optimizations (保持性能，提升速度):")
print("- Increased learning rate (0.18) for faster convergence")
print("- Reduced max iterations based on log analysis (282轮停止)")
print("- Fixed feature_pre_filter warning")
print("- Tighter early stopping for efficiency")

lgb_model = lgb.train(
    final_lgb_params,
    lgb_train,
    num_boost_round=1000,        # 从2000减少到1000 (基于日志282轮停止)
    valid_sets=[lgb_train, lgb_val],
    callbacks=[lgb.early_stopping(80), lgb.log_evaluation(50)]  # 从130减少到80
)


Training LightGBM model with v2 business-optimized parameters...
V2 business feature optimizations:
- Reduced leaves (140) and depth (10) to prevent overfitting
- Lower learning rate (0.15) for better stability
- Increased regularization (L1=0.006, L2=7.5) for generalization
- Conservative sampling for better model robustness


Training until validation scores don't improve for 130 rounds


[50]	training's ndcg@3: 0.838298	valid_1's ndcg@3: 0.839591

[100]	training's ndcg@3: 0.858681	valid_1's ndcg@3: 0.841336


[150]	training's ndcg@3: 0.873883	valid_1's ndcg@3: 0.841788

...

[250]	training's ndcg@3: 0.894915	valid_1's ndcg@3: 0.841154


[300]	training's ndcg@3: 0.903706	valid_1's ndcg@3: 0.842088

[350]	training's ndcg@3: 0.911544	valid_1's ndcg@3: 0.840891

[400]	training's ndcg@3: 0.918839	valid_1's ndcg@3: 0.840583

Early stopping, best iteration is:
[282]	training's ndcg@3: 0.900754	valid_1's ndcg@3: 0.842607

In [ ]:
print("\n--- Training LightGBM DART Model (v3 - Efficiency Optimized) ---")

# DART参数优化：基于日志分析大幅减少训练轮数
dart_params = {
    'objective': 'lambdarank', 
    'metric': 'ndcg', 
    'eval_at': [3],
    'boosting_type': 'dart', 
    'n_estimators': 1200,      # 从2500大幅减少到1200 (基于日志分析)
    'learning_rate': 0.055,    # 从0.045提升到0.055，更快收敛
    'num_leaves': 65,          # 从60增加到65，稍微增加容量
    'drop_rate': 0.08,         # 从0.075增加到0.08，更好的正则化
    'subsample': 0.87,         # 从0.85增加到0.87，更好的采样
    'skip_drop': 0.45,         # 从0.48减少到0.45，更多dropout
    'max_depth': 12,           # 保持12，控制复杂度
    'min_child_samples': 45,   # 保持45，防止过拟合
    'lambda_l1': 0.002,        # 从0.001增加到0.002，更多L1正则化
    'lambda_l2': 3.0,          # 从2.5增加到3.0，更多L2正则化
    'feature_pre_filter': False, # 修复LightGBM警告
    'n_jobs': -1,
    'random_state': RANDOM_STATE,
    'label_gain': [0, 1]
}

print("\nTraining Enhanced LightGBM DART model (v3 - 效率优化)...")
print("V3 DART efficiency optimizations (保持性能，大幅提升速度):")
print("- Reduced iterations from 2500 to 1200 (基于日志2400+轮分析)")
print("- Increased learning rate (0.055) for faster convergence") 
print("- Enhanced regularization to compensate for fewer iterations")
print("- Fixed feature_pre_filter warning")
print("- Target: Same performance with 50% less training time")

lgb_model_dart = lgb.train(
    dart_params,
    lgb_train, 
    num_boost_round=dart_params['n_estimators'], 
    valid_sets=[lgb_val],
    callbacks=[lgb.log_evaluation(50)]  # DART不支持early stopping，但减少了总轮数
)

print("\n🎉 LightGBM models training completed successfully!")
print("Note: CatBoost training moved to dedicated section below for proper data handling.")


--- Training LightGBM DART Model ---

Training Enhanced LightGBM DART model (v2)...
V2 DART optimizations:
- Refined learning rate (0.045) for better convergence
- Increased capacity (60 leaves) with depth control (12)
- Optimized DART dropout balance (0.075 rate, 0.48 skip)
- Added regularization for better generalization


[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.


[LightGBM] [Fatal] Reducing `min_data_in_leaf` with `feature_pre_filter=true` may cause unexpected behaviour for features that were pre-filtered by the larger `min_data_in_leaf`.
You need to set `feature_pre_filter=false` to dynamically change the `min_data_in_leaf`.


/usr/local/lib/python3.10/site-packages/lightgbm/callback.py:333: UserWarning: Early stopping is not available in dart mode
  _log_warning("Early stopping is not available in dart mode")


[50]	valid_0's ndcg@3: 0.826599

...

[2200]	valid_0's ndcg@3: 0.844405


[2250]	valid_0's ndcg@3: 0.844383


[2300]	valid_0's ndcg@3: 0.844701


[2350]	valid_0's ndcg@3: 0.844149


[2400]	valid_0's ndcg@3: 0.844231


## 4. Blending and Final Evaluation

In [ ]:
print("\n--- Training CatBoost Model (CORRECTED v3) ---")
print("Training CatBoost with FIXED DataFrame approach and v3 efficiency optimizations...")

# Prepare data for CatBoost (needs specific format)
from catboost import CatBoostRanker, Pool

# CRITICAL FIX: Use DataFrame approach instead of numpy arrays for mixed types
# The problem: numpy arrays can't handle mixed int/float types properly for CatBoost
# Solution: Use pandas DataFrames which preserve column-specific data types

print("Preparing CatBoost data with FIXED DataFrame approach...")

# Step 1: Get data splits using the same indices as other models
X_catboost_tr = X[:n1].clone()  # Training split
X_catboost_va = X[n1:n2].clone()  # Validation split 
X_catboost_te = X[n2:].clone()  # Test split

print(f"CatBoost data splits: Train={len(X_catboost_tr)}, Val={len(X_catboost_va)}, Test={len(X_catboost_te)}")

# Step 2: CRITICAL FIX - Apply proper categorical encoding
# Convert categorical features to integers using a unified approach
print(f"Converting {len(cat_features_final)} categorical features to integers...")

# Create a unified string-to-integer mapping for each categorical feature
for col in cat_features_final:
    print(f"   Processing categorical feature: {col}")
    
    # Combine all data to create consistent encoding across splits
    all_values = pl.concat([
        X_catboost_tr.select(col),
        X_catboost_va.select(col), 
        X_catboost_te.select(col)
    ]).unique().sort(col)
    
    # Create mapping: unique values -> integers (0, 1, 2, ...)
    mapping_dict = {
        val: idx for idx, val in enumerate(all_values[col].to_list())
    }
    
    # Apply mapping to all splits consistently
    X_catboost_tr = X_catboost_tr.with_columns(
        pl.col(col).map_elements(lambda x: mapping_dict.get(x, -1), return_dtype=pl.Int32).alias(col)
    )
    X_catboost_va = X_catboost_va.with_columns(
        pl.col(col).map_elements(lambda x: mapping_dict.get(x, -1), return_dtype=pl.Int32).alias(col)
    )
    X_catboost_te = X_catboost_te.with_columns(
        pl.col(col).map_elements(lambda x: mapping_dict.get(x, -1), return_dtype=pl.Int32).alias(col)
    )

# Step 3: Convert numerical features to float32 for memory efficiency
non_cat_features = [col for col in feature_cols if col not in cat_features_final]
print(f"Converting {len(non_cat_features)} numerical features to float32...")

for col in non_cat_features:
    X_catboost_tr = X_catboost_tr.with_columns(pl.col(col).cast(pl.Float32).alias(col))
    X_catboost_va = X_catboost_va.with_columns(pl.col(col).cast(pl.Float32).alias(col))
    X_catboost_te = X_catboost_te.with_columns(pl.col(col).cast(pl.Float32).alias(col))

print("CatBoost data preparation completed with FIXED data types")

# Step 4: Convert to pandas DataFrames (CRITICAL FIX for mixed types)
print("Converting to pandas DataFrames for CatBoost compatibility...")

X_catboost_tr_df = X_catboost_tr.to_pandas()
X_catboost_va_df = X_catboost_va.to_pandas()
X_catboost_te_df = X_catboost_te.to_pandas()

# Verify data types before Pool creation
print("Verifying data types before Pool creation...")
print(f"   Training data shape: {X_catboost_tr_df.shape}")
print(f"   Training data memory usage: {X_catboost_tr_df.memory_usage().sum() / 1024**2:.1f} MB")

# Check categorical columns specifically
for i, cat_idx in enumerate(catboost_cat_indices[:3]):
    col_name = X_catboost_tr_df.columns[cat_idx]
    col_data = X_catboost_tr_df.iloc[:, cat_idx]
    print(f"   Cat feature {i} (col {cat_idx}, '{col_name}'): dtype={col_data.dtype}, sample={col_data.head(3).tolist()}")

# Step 5: Create CatBoost pools with properly typed DataFrames
print("Creating CatBoost Pool objects with DataFrames...")

train_pool = Pool(
    data=X_catboost_tr_df,  # ✅ Using DataFrame instead of numpy array
    label=y_tr.to_numpy().flatten(),
    group_id=groups_tr.to_numpy().flatten(),
    cat_features=catboost_cat_indices
)

val_pool = Pool(
    data=X_catboost_va_df,  # ✅ Using DataFrame instead of numpy array
    label=y_va.to_numpy().flatten(),
    group_id=groups_va.to_numpy().flatten(),
    cat_features=catboost_cat_indices
)

print("CatBoost Pool objects created successfully!")
print(f"   Training pool: {train_pool.num_row()} rows x {train_pool.num_col()} cols")
print(f"   Validation pool: {val_pool.num_row()} rows x {val_pool.num_col()} cols")
print(f"   Categorical features: {len(catboost_cat_indices)} indices")

# Step 6: Train CatBoost model with v3 efficiency-optimized parameters
print("Training CatBoost model with v3 efficiency optimizations...")

# CatBoost parameters optimized for business features (v3 - Efficiency)
catboost_params = {
    'loss_function': 'YetiRank',
    'custom_metric': ['NDCG:top=3'],
    'iterations': 1500,          # 从2000减少到1500，基于其他模型收敛分析
    'learning_rate': 0.15,       # 从0.12提升到0.15，更快收敛
    'depth': 8,                  # 保持8，良好的深度
    'l2_leaf_reg': 4.0,          # 保持4.0，适度正则化
    'bootstrap_type': 'Bayesian',
    'bagging_temperature': 0.8,
    'subsample': 0.85,
    'random_strength': 0.8,
    'one_hot_max_size': 10,
    'max_ctr_complexity': 3,
    'random_seed': RANDOM_STATE,
    'thread_count': -1,
    'verbose': 50
}

print("V3 CatBoost efficiency optimizations:")
print("- Reduced iterations from 2000 to 1500")
print("- Increased learning rate (0.15) for faster convergence")
print("- Tighter early stopping for efficiency")
print("- FIXED DataFrame approach prevents data type errors")
print("- Target: Same performance with 25% less training time")

catboost_model = CatBoostRanker(**catboost_params)
catboost_model.fit(
    train_pool, 
    eval_set=val_pool,
    early_stopping_rounds=80,    # 从100减少到80，更激进的停止
    verbose=50
)

print("CatBoost model trained successfully with FIXED DataFrame approach!")
print("All models training completed successfully!")

### 🔧 CatBoost Data Type Fix

**问题**: CatBoost在使用numpy数组时，所有数据会被转换为float64，导致分类特征类型冲突错误：
```
CatBoostError: 'data' is numpy array of floating point numerical type, 
but 'cat_features' parameter specifies nonzero number of categorical features
```

**解决方案**: 
1. ✅ 使用pandas DataFrame而不是numpy数组
2. ✅ 确保分类特征保持int32类型
3. ✅ 数值特征使用float32类型
4. ✅ DataFrame能正确处理混合数据类型

**注意**: 这个修复确保CatBoost能正确识别和处理分类特征，避免训练时的类型错误。

## 5. Submission

In [ ]:
# Validation and Ensemble Optimization (Required Variables)
print("Setting up ensemble strategies...")

# Get validation predictions for ensemble optimization
xgb_val_preds = xgb_model.predict(dval)
lgb_gbdt_val_preds = lgb_model.predict(data_xgb_va)
lgb_dart_val_preds = lgb_model_dart.predict(data_xgb_va)

# CatBoost validation predictions
catboost_val_preds = catboost_model.predict(val_pool)

# Calculate individual model performance on validation set
val_hitrates = {}
val_hitrates['XGBoost'] = hitrate_at_3(y_va.to_numpy().flatten(), xgb_val_preds, groups_va.to_numpy().flatten())
val_hitrates['LightGBM_GBDT'] = hitrate_at_3(y_va.to_numpy().flatten(), lgb_gbdt_val_preds, groups_va.to_numpy().flatten())
val_hitrates['LightGBM_DART'] = hitrate_at_3(y_va.to_numpy().flatten(), lgb_dart_val_preds, groups_va.to_numpy().flatten())
val_hitrates['CatBoost'] = hitrate_at_3(y_va.to_numpy().flatten(), catboost_val_preds, groups_va.to_numpy().flatten())

print("Individual model validation performance:")
for model, hr in val_hitrates.items():
    print(f"  {model}: {hr:.4f}")

# Define ensemble strategies
strategies = {
    "Static Balanced": np.array([0.35, 0.25, 0.10, 0.30]),  # XGB, LGB-GBDT, LGB-DART, CatBoost
    "DART Focused": np.array([0.25, 0.25, 0.35, 0.15]),
    "Performance Weighted": np.array([0.4, 0.3, 0.1, 0.2]),
    "Static XGBoost Focused": np.array([0.45, 0.15, 0.05, 0.35])
}

# Test strategies on validation data
val_submission_df = data_xgb_va.select(['ranker_id']).with_columns([
    pl.Series('xgb_score', xgb_val_preds),
    pl.Series('lgb_gbdt_score', lgb_gbdt_val_preds),
    pl.Series('lgb_dart_score', lgb_dart_val_preds),
    pl.Series('catboost_score', catboost_val_preds)
])

# Convert scores to ranks
val_submission_df = val_submission_df.with_columns([
    pl.col("xgb_score").rank(method="average", descending=True).over("ranker_id").alias("xgb_rank"),
    pl.col("lgb_gbdt_score").rank(method="average", descending=True).over("ranker_id").alias("lgb_gbdt_rank"),
    pl.col("lgb_dart_score").rank(method="average", descending=True).over("ranker_id").alias("lgb_dart_rank"),
    pl.col("catboost_score").rank(method="average", descending=True).over("ranker_id").alias("catboost_rank")
])

# Test ensemble strategies
best_strategy_hr3 = 0
best_strategy_name = "Static Balanced"
strategy_results = {}

for strategy_name, weights in strategies.items():
    # Create ensemble
    ensemble_score = (weights[0] * val_submission_df.get_column("xgb_rank") + 
                     weights[1] * val_submission_df.get_column("lgb_gbdt_rank") + 
                     weights[2] * val_submission_df.get_column("lgb_dart_rank") + 
                     weights[3] * val_submission_df.get_column("catboost_rank"))
    
    # Get best option per group (lowest ensemble rank)
    val_ensemble_df = val_submission_df.with_columns([
        pl.Series("ensemble_rank", ensemble_score)
    ])
    
    # Rank ensemble scores within each group
    val_ensemble_df = val_ensemble_df.with_columns([
        pl.col("ensemble_rank").rank(method="ordinal", descending=False).over("ranker_id").alias("final_rank")
    ])
    
    # Select top option per group
    selected = val_ensemble_df.filter(pl.col("final_rank") == 1)
    
    # Create predictions (1 for selected, 0 for others)
    val_preds = np.zeros(len(val_ensemble_df))
    selected_indices = selected.select(pl.int_range(pl.len()).over("ranker_id")).to_numpy().flatten()
    # val_preds[selected_indices] = 1
    
    # Calculate hit rate using ensemble ranking
    val_ensemble_preds = val_ensemble_df.get_column("ensemble_rank").to_numpy()
    strategy_hr3 = hitrate_at_3(y_va.to_numpy().flatten(), -val_ensemble_preds, groups_va.to_numpy().flatten())
    
    strategy_results[strategy_name] = strategy_hr3
    print(f"Strategy '{strategy_name}': {strategy_hr3:.4f}")
    
    if strategy_hr3 > best_strategy_hr3:
        best_strategy_hr3 = strategy_hr3
        best_strategy_name = strategy_name

# Set the best strategy weights
if best_strategy_name in strategies:
    optimized_weights = strategies[best_strategy_name]
    dart_focused_weights = strategies.get("DART Focused", strategies["Static Balanced"])
    performance_weights = strategies.get("Performance Weighted", strategies["Static Balanced"])
    balanced_weights = strategies["Static Balanced"]
else:
    optimized_weights = strategies["Static Balanced"]
    dart_focused_weights = strategies["Static Balanced"]
    performance_weights = strategies["Static Balanced"]
    balanced_weights = strategies["Static Balanced"]

print(f"\nBest strategy: {best_strategy_name} (HR@3: {best_strategy_hr3:.4f})")
print("Ensemble optimization completed successfully!")

In [ ]:
print("Generating predictions on test set...")

# XGBoost test predictions
xgb_test_preds = xgb_model.predict(dtest)

# LightGBM test predictions  
lgb_gbdt_test_preds = lgb_model.predict(data_xgb_te)
lgb_dart_test_preds = lgb_model_dart.predict(data_xgb_te)

# CatBoost test predictions with FIXED DataFrame approach
print("Preparing CatBoost test predictions with FIXED DataFrame approach...")

# CatBoost test data is already properly encoded in the training cell
# X_catboost_te was processed with the same categorical encoding pipeline
# CRITICAL FIX: Use DataFrame instead of numpy array for test pool

test_pool = Pool(
    data=X_catboost_te_df,  # Use the DataFrame version from training cell
    group_id=groups_te.to_numpy().flatten(),
    cat_features=catboost_cat_indices
)

print("CatBoost test pool created successfully with DataFrame approach")
catboost_test_preds = catboost_model.predict(test_pool)
print("CatBoost test predictions generated successfully")

# Create comprehensive submission dataframe with all models and strategies
submission_df = test.select(['Id', 'ranker_id']).with_columns([
    pl.Series('xgb_score', xgb_test_preds),
    pl.Series('lgb_gbdt_score', lgb_gbdt_test_preds),
    pl.Series('lgb_dart_score', lgb_dart_test_preds),
    pl.Series('catboost_score', catboost_test_preds)
])

In [ ]:
# Convert model scores to ranks within each group (Required for submission)
print("Converting model scores to ranks for ensemble strategies...")

submission_df = submission_df.with_columns([
    pl.col("xgb_score").rank(method="average", descending=True).over("ranker_id").alias("xgb_rank"),
    pl.col("lgb_gbdt_score").rank(method="average", descending=True).over("ranker_id").alias("lgb_gbdt_rank"),
    pl.col("lgb_dart_score").rank(method="average", descending=True).over("ranker_id").alias("lgb_dart_rank"),
    pl.col("catboost_score").rank(method="average", descending=True).over("ranker_id").alias("catboost_rank")
])

# Apply best ensemble strategy from validation
print(f"Applying best ensemble strategy: {best_strategy_name}")
ensemble_rank = (optimized_weights[0] * submission_df.get_column("xgb_rank") + 
                optimized_weights[1] * submission_df.get_column("lgb_gbdt_rank") + 
                optimized_weights[2] * submission_df.get_column("lgb_dart_rank") + 
                optimized_weights[3] * submission_df.get_column("catboost_rank"))

# Create final ranking within each group (1 = best, 2 = second best, etc.)
submission_df = submission_df.with_columns([
    pl.Series("ensemble_rank", ensemble_rank)
]).with_columns([
    pl.col("ensemble_rank").rank(method="ordinal", descending=False).over("ranker_id").alias("selected")
])

# Create final submission with required format: Id, ranker_id, selected
final_submission = submission_df.select(['Id', 'ranker_id', 'selected'])

print("Final submission format validation...")
print(f"Submission shape: {final_submission.shape}")
print(f"Unique ranker_ids: {final_submission['ranker_id'].n_unique()}")
print(f"Rank range: {final_submission['selected'].min()} to {final_submission['selected'].max()}")

# Validate submission format (Critical for competition)
validation_results = final_submission.group_by('ranker_id').agg([
    pl.col('selected').min().alias('min_rank'),
    pl.col('selected').max().alias('max_rank'),
    pl.col('selected').n_unique().alias('unique_ranks'),
    pl.col('selected').len().alias('group_size')
])

# Check if all groups have valid permutations (1, 2, 3, ..., N)
invalid_groups = validation_results.filter(
    (pl.col('min_rank') != 1) | 
    (pl.col('max_rank') != pl.col('group_size')) |
    (pl.col('unique_ranks') != pl.col('group_size'))
)

if len(invalid_groups) > 0:
    print(f"⚠️  WARNING: Found {len(invalid_groups)} groups with invalid rankings!")
    print("Sample invalid groups:")
    print(invalid_groups.head())
else:
    print("✅ All groups have valid rank permutations (1, 2, 3, ..., N)")

# Display sample submission
print("\nSample submission format:")
print(final_submission.head(10))

# Verify submission requirements
print("\n=== SUBMISSION FORMAT VALIDATION ===")
print(f"✅ Required columns: {list(final_submission.columns) == ['Id', 'ranker_id', 'selected']}")
print(f"✅ All ranks are integers: {final_submission['selected'].dtype in [pl.Int32, pl.Int64]}")
print(f"✅ All ranks ≥ 1: {final_submission['selected'].min() >= 1}")
print(f"✅ Total rows: {len(final_submission):,}")
print(f"✅ Row order preserved: Test set order maintained")

# Save submission file
submission_file = "submission.csv"
final_submission.write_csv(submission_file)
print(f"\n🎯 Submission saved to: {submission_file}")
print("Ready for Kaggle submission!")

In [ ]:
# COMPREHENSIVE SUBMISSION VALIDATION (Critical for competition success)
print("=== COMPREHENSIVE SUBMISSION VALIDATION ===")

# 1. Load and verify test set order preservation
print("1. Verifying row order preservation...")
test_ids = test.select('Id').to_numpy().flatten()
submission_ids = final_submission.select('Id').to_numpy().flatten()
order_preserved = np.array_equal(test_ids, submission_ids)
print(f"   ✅ Row order preserved: {order_preserved}")
if not order_preserved:
    print("   ❌ CRITICAL ERROR: Row order not preserved!")
    print(f"   Test IDs: {test_ids[:5]}...")
    print(f"   Submission IDs: {submission_ids[:5]}...")

# 2. Detailed rank validation per group
print("2. Validating rank permutations...")
rank_validation = final_submission.group_by('ranker_id').agg([
    pl.col('selected').sort().alias('sorted_ranks'),
    pl.col('selected').len().alias('n_options')
]).with_columns([
    pl.col('sorted_ranks').list.eval(pl.int_range(1, pl.col('n_options') + 1)).alias('expected_ranks'),
    (pl.col('sorted_ranks').list.eval(pl.int_range(1, pl.col('n_options') + 1)) == pl.col('sorted_ranks')).alias('valid_permutation')
])

invalid_count = rank_validation.filter(~pl.col('valid_permutation')).height
print(f"   ✅ Valid permutations: {rank_validation.height - invalid_count}/{rank_validation.height}")
if invalid_count > 0:
    print(f"   ❌ Invalid permutations found: {invalid_count}")
    invalid_sample = rank_validation.filter(~pl.col('valid_permutation')).head(3)
    print("   Sample invalid groups:")
    print(invalid_sample)

# 3. Check for duplicate ranks within groups
print("3. Checking for duplicate ranks...")
duplicate_check = final_submission.group_by('ranker_id').agg([
    pl.col('selected').len().alias('total_ranks'),
    pl.col('selected').n_unique().alias('unique_ranks')
]).with_columns([
    (pl.col('total_ranks') == pl.col('unique_ranks')).alias('no_duplicates')
])

duplicates_found = duplicate_check.filter(~pl.col('no_duplicates')).height
print(f"   ✅ No duplicate ranks: {duplicates_found == 0}")
if duplicates_found > 0:
    print(f"   ❌ Groups with duplicate ranks: {duplicates_found}")

# 4. Sample group examination
print("4. Examining sample groups...")
sample_groups = final_submission.group_by('ranker_id').agg([
    pl.col('Id').alias('ids'),
    pl.col('selected').alias('ranks')
]).head(3)

for i, row in enumerate(sample_groups.iter_rows()):
    ranker_id, ids, ranks = row
    print(f"   Group {i+1} (ranker_id: {ranker_id}):")
    print(f"     IDs: {ids}")
    print(f"     Ranks: {ranks}")
    print(f"     Valid: {sorted(ranks) == list(range(1, len(ranks) + 1))}")

# 5. Final submission statistics
print("5. Final submission statistics...")
submission_stats = final_submission.select([
    pl.col('ranker_id').n_unique().alias('unique_groups'),
    pl.col('Id').len().alias('total_rows'),
    pl.col('selected').min().alias('min_rank'),
    pl.col('selected').max().alias('max_rank'),
    pl.col('selected').mean().alias('avg_rank')
])

print(f"   Total groups: {submission_stats['unique_groups'].item():,}")
print(f"   Total rows: {submission_stats['total_rows'].item():,}")
print(f"   Rank range: {submission_stats['min_rank'].item()} to {submission_stats['max_rank'].item()}")
print(f"   Average rank: {submission_stats['avg_rank'].item():.2f}")

# 6. Competition format compliance check
print("6. Competition format compliance...")
required_columns = ['Id', 'ranker_id', 'selected']
actual_columns = final_submission.columns
format_compliant = (actual_columns == required_columns)
print(f"   ✅ Required columns {required_columns}: {format_compliant}")
print(f"   ✅ Integer ranks: {final_submission['selected'].dtype in [pl.Int32, pl.Int64]}")
print(f"   ✅ No missing values: {final_submission.null_count().sum_horizontal().item() == 0}")

# 7. Save verification
import os
if os.path.exists(submission_file):
    file_size = os.path.getsize(submission_file) / 1024 / 1024  # MB
    print(f"   ✅ File saved: {submission_file} ({file_size:.2f} MB)")
else:
    print(f"   ❌ File not found: {submission_file}")

print("\n" + "="*50)
if order_preserved and invalid_count == 0 and duplicates_found == 0 and format_compliant:
    print("🎉 SUBMISSION VALIDATION PASSED!")
    print("✅ Your submission meets all competition requirements")
    print("✅ Ready for Kaggle upload")
else:
    print("⚠️  SUBMISSION VALIDATION FAILED!")
    print("❌ Please fix the issues above before submitting")
print("="*50)